## Imports

In [2]:
import cv2
import cupy as cp
import numpy as np

from skimage.metrics import structural_similarity as ssim
from matplotlib import pyplot as plt
from cupyx.scipy.signal import fftconvolve
from cupyx.scipy.spatial.distance import cdist

## Helper Functions

In [3]:
def convolve_arrays(array1: np.ndarray, array2: np.ndarray, mode='same') -> cp.ndarray:
    cupy_array1 = cp.asarray(array1)
    cupy_array2 = cp.asarray(array2)
    return fftconvolve(cupy_array1, cupy_array2, mode=mode)

In [4]:
def gaussian_filter(size, sigma):
    x, y = np.meshgrid(np.linspace(-1, 1, size), np.linspace(-1, 1, size))
    d = np.sqrt(x*x + y*y)
    g = np.exp(-(d**2 / (2.0 * sigma**2)))
    return g / g.sum()

In [5]:
def MSE(arr1, arr2):
    mse = cp.mean((arr1 - arr2) ** 2)
    return mse

In [6]:
def kernel_and_divide(sigma, size, le_input, f_name):
    gaussian = gaussian_filter(size, sigma)
    image_le_temp = convolve_arrays(le_input, gaussian)
    division = cp.divide(cp.asnumpy(le_input), image_le_temp)
    plt.imsave("f_name", cp.asnumpy(division), cmap="gray")

In [7]:
def sift_similarity(img1_path, img2_path):
    # Read images
    img1 = cv2.imread(img1_path, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img2_path, cv2.IMREAD_GRAYSCALE)

    # Initialize SIFT detector
    sift = cv2.SIFT_create()

    # Detect and compute SIFT features
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    # Convert descriptors to CuPy arrays for faster computation
    des1_gpu = cp.asarray(des1)
    des2_gpu = cp.asarray(des2)

    # Compute pairwise distances between descriptors
    distances = cdist(des1_gpu, des2_gpu, metric='euclidean')

    # Convert distances back to numpy for CPU operations
    distances_cpu = cp.asnumpy(distances)

    # Find the best matches
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # Calculate similarity score
    similarity_score = len(good_matches) / min(len(kp1), len(kp2))

    return similarity_score

# Example usage
img1_path = 'good.jpg'
img2_path = 'Prostate.jpg'
similarity = sift_similarity(img1_path, img2_path)
print(f"Similarity score: {similarity}")

RuntimeError: pylibraft is not installed

## Algo